In [11]:
import pandas as pd
from Bio import SeqIO
from Bio import Seq
import numpy as np

DE_NOVO_ASSEMBLE_CHROM_DIR = '/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/de_novo_assemble/'


Reading in k-mer sequences
Writing fastq files...
Written to  /home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/de_novo_assemble/test.fasta


In [42]:
c = 20
localized = pd.read_table('/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/unmapped/localized.tsv', comment='#')
localized.columns = ['# median_of_nonzeros', 'prevelance', 'chrom_pred', 'start_pred', 'end_pred']
localized.chrom_pred = [np.nan if np.isnan(i) else round(i) for i in localized.chrom_pred ]
print(100*np.mean(~np.isnan(localized.chrom_pred)),'% localized')
localized = localized[~np.isnan(localized.chrom_pred)]
l = localized[localized.chrom_pred==c].sort_values(['start_pred', 'end_pred'])[['start_pred', 'end_pred']]


93.73570065538608 % localized


In [43]:
print("Reading in k-mer sequences")
kmer_seqs = pd.read_table('/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/family_likelihoods/kmers_unmapped_prev_and_median_filt.txt', 
                          header=None)
print("Writing fastq files...")
sequences = [SeqIO.SeqRecord(seq = Seq.Seq(seq[0]), id=str(idx)) for idx, seq in kmer_seqs.iloc[l.index].iterrows()]
SeqIO.write(sequences, DE_NOVO_ASSEMBLE_CHROM_DIR + "test.fasta", "fasta")
print('Written to ', DE_NOVO_ASSEMBLE_CHROM_DIR + "test.fasta" )

Reading in k-mer sequences
Writing fastq files...
Written to  /home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/de_novo_assemble/test.fasta


In [65]:
len(sequences)

184175

In [68]:
%%bash
chrom=20
#for file in /home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/de_novo_assemble/chr${chrom}/*.fasta ; do
file=c
echo "ASSEMBLING" ${file}
assembled_dir="${file/.fasta/_assembled}"
# Remove previous directory to start clean.
\rm ${assembled_dir} -r    
# Assemble.
megahit -r $file -o ${assembled_dir} --k-list=51 --no-mercy --min-count=1
#done

ASSEMBLING c


rm: cannot remove ‘c’: No such file or directory
2021-09-21 17:28:40 - MEGAHIT v1.2.9
2021-09-21 17:28:40 - Using megahit_core with POPCNT and BMI2 support
2021-09-21 17:28:40 - Convert reads to binary library
2021-09-21 17:28:40 - b'INFO  sequence/io/sequence_lib.cpp  :   77 - Lib 0 (/home/groups/dpwall/briannac/alt_haplotypes/src/de_novo_assemble/c): se, 0 reads, 0 max length'
2021-09-21 17:28:40 - b'INFO  utils/utils.h                 :  152 - Real: 0.1746\tuser: 0.0005\tsys: 0.0015\tmaxrss: 7268'
2021-09-21 17:28:40 - Start assembly. Number of CPU threads 128 
2021-09-21 17:28:40 - k list: 51 
2021-09-21 17:28:40 - Memory used: 973408186368
2021-09-21 17:28:40 - Extracting solid (k+1)-mers and building sdbg for k = 51
2021-09-21 17:28:41 - Assemble contigs from SdBG for k = 51
2021-09-21 17:28:41 - Merging to output final contigs 
2021-09-21 17:28:41 - 0 contigs, total 0 bp, min 0 bp, max 0 bp, avg 0 bp, N50 0 bp
2021-09-21 17:28:42 - ALL DONE. Time elapsed: 1.326979 seconds 


In [46]:
assembled = [r for r in SeqIO.parse(DE_NOVO_ASSEMBLE_CHROM_DIR + "test_assembled/final.contigs.fa", "fasta")]

In [58]:
is_in_seq = np.array([[str(k.seq) in str(a.seq) for k in sequences] for a in assembled[:10]])

In [60]:
len(sequences)

184175

In [59]:
is_in_seq.sum()

33

In [56]:
k.seq

NameError: name 'k' is not defined

In [52]:
sequences

[SeqRecord(seq=Seq('AATGGAATGGAAGGGAAGGGAATAGAATCAAACCAAGTGCATTGGAATGGAATG...TGG'), id='151918', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('CCCTCCCCCCACCCAACCACCCCCTGACGCAGGCCCCGACGTCCTGGGATCCTT...AGG'), id='1596057', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('ATACAATATGACATATAGTATATATTATAATTTATGACATATATAATATATGGT...AAA'), id='3102602', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('ATTTATTATATATAATATAATACATGTCATATATTGTATATAATTTATAATATA...ATG'), id='3194366', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('AAATGAAAATATGTAGAATTACATGTTTATCTAGTTATATGTAAAAAATATAGT...ATA'), id='5487015', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('CACTCGGGTTGATTCCATTCCATTCCATTAAATTGTATTCCGTTCCTTTCCATG...TTC'), id='930153', name='<unknown name>', description='<unknown desc